# GENA-LM Sequence classification example
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AIRI-Institute/GENA_LM/blob/main/notebooks/GENA_sequence_classification_example.ipynb)

In this notebook, we will fine-tune pre-trained GENA-LM model on promoter prediction task. This task is formulated as a sequence classification: predict one of the labels based on the input sequence.

## Install requirements
We would need PyTorch, HuggingFace Transformers and Datasets.

In [ ]:
! pip install torch --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
! pip install transformers[torch] datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.3 MB/s eta 0:00:00


Check that PyTorch has been successfully installed. Make sure we have access to GPU (the following code should return `True`).

In [ ]:
import torch
torch.cuda.is_available()

True

## Get pre-trained GENA-LM model

We will use HuggingFace Transformer library to download pre-trained GENA-LM model.

Table of available pre-trained GENA-LM models:
https://github.com/AIRI-Institute/GENA_LM#pre-trained-models

### Pre-trained GENA-LM for Masked Language Modeling
Let's begin with downloading the GENA-LM model pre-trained on Masked Language Modeling task. We would also need a tokenizer to convert an input sequence in nucleotides into a sequence of tokens.

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained('AIRI-Institute/gena-lm-bert-base-t2t')
model = AutoModel.from_pretrained('AIRI-Institute/gena-lm-bert-base-t2t', trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/AIRI-Institute/gena-lm-bert-base-t2t:
- modeling_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Let's check how model looks like:

In [ ]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=3)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (pre_attention_ln): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (post_attention_ln): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
              (softmax): Sof

We can see that `AIRI-Institute/gena-lm-bert-base-t2t` has an embedding layer that converts tokens to vectors of size 768, then 12 Transformer layers (0-11 BertLayer) and finally a task-specific head, in this case `BertOnlyMLMHead`.

We can use this model to extract sequence or tokens embeddings from different layers. Let's do this with an example sequence.

In [ ]:
seq = 'CACCCAGAGAGAGTAACCAGAATGGATACATTTTGGCCAACATGATTCTAACCCAGTGAGACCCATTTTGGGCTTATG'

Apply tokenizer to convert sequence of nucleotides to sequence of tokens. We also add special tokens `[CLS]` at the beginning of the sequence and `[SEP]` at the end. In case, if we want to solve sequence-level tasks, the output of Transformer for `[CLS]` token is usually used as input to task-specific heads.
If we want to solve token-level tasks, we can collect model outputs for each token. `[SEP]` token is used to separate different sequences in the input.

In [ ]:
tokens = tokenizer.tokenize(seq, add_special_tokens=True)
print('tokens:', tokens)
print('n_tokens:', len(tokens))

tokens: ['[CLS]', 'CACCC', 'AGAGAGAG', 'TAACC', 'AGAATGG', 'ATACATT', 'TTGGCC', 'AACATG', 'ATTC', 'TAACCC', 'AGTGAGACCC', 'ATTTTGGGC', 'TTATG', '[SEP]']
n_tokens: 14


Let's feed tokenized sequence to the model and check it outputs:

In [ ]:
with torch.no_grad():
  output = model(**tokenizer(seq, return_tensors='pt'), output_hidden_states=True)
print(output.keys())

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:900: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


odict_keys(['logits', 'hidden_states'])


The model returns dict with `hidden_states` and `logits`. `hidden_states` contains the model hidden states of the model on each layer. `logits` corresponds to the output of task-specific head, in our case MLM.

Let's check the output of the last layer:

In [ ]:
output['hidden_states'][-1].shape

torch.Size([1, 14, 768])

We have 14 hidden states (embeddings) of size 768. Each hidden state corresponds to a single input token. Let's take embedding for the `[CLS]` token from the last layer. `[CLS]` token has index 0:

In [ ]:
output['hidden_states'][-1][0][0]

tensor([ 1.7935e+00,  9.7426e+00, -2.8164e-01, -1.0098e+01,  2.3238e+01,
        -3.0767e+00,  1.0525e+00, -7.9911e+00, -3.1323e+00,  1.0583e+01,
         1.0248e+01, -3.4555e+00,  1.1753e+01,  1.4058e+00,  1.2386e+01,
        -1.7454e+00, -2.3567e+01, -1.7231e+01, -8.1573e+00,  1.2149e+01,
         4.2525e+00, -8.0962e+00,  2.3365e-01,  1.9836e+01, -5.4546e+00,
        -3.0513e+00, -1.3787e+01, -5.8350e+00,  2.2220e+01,  1.5292e+01,
        -9.6476e+00,  2.0491e+01, -1.1205e+01,  1.0240e+01, -4.2015e+00,
         7.7502e+00, -3.5554e+00,  3.6108e+00, -8.0955e+00,  1.7012e-01,
        -1.0645e+00, -3.8739e+00,  1.6354e+01, -1.0127e+01, -1.1638e+01,
         3.9613e-01,  3.2580e+01,  1.7921e+01,  1.0905e+01,  4.1827e+00,
        -1.0156e+01, -3.2742e+00, -1.4447e+01,  1.9717e+01,  5.1663e+00,
         1.1061e+01, -1.0057e+00,  2.0811e+01,  2.6003e+01, -1.9037e+01,
         1.2134e+01,  2.5108e+01,  3.6361e+00, -1.7838e+01,  6.4099e+00,
         2.0844e-01,  2.0902e+01,  8.4496e+00, -6.0

### Pre-trained GENA-LM to finetune on sequence classification task
To fine-tune the pre-trained GENA-LM, we need to initialize the model with the sequence classification head. Since GENA-LM does not know anything about the task, this head is randomly initialized. During fine-tuning we will train this head and update the weights of other GENA layers to solve our task. However, another possible approach is to freeze (do not update) all parameters except the head.

#### with HuggingFace
We need to get model class `cls` that we want to use in our task: `BertForSequenceClassification`

In [ ]:
gena_module_name = model.__class__.__module__
print(gena_module_name)

transformers_modules.AIRI-Institute.gena-lm-bert-base-t2t.21343b983208dd7bd3430f5a0d812ab6131faa7d.modeling_bert


In [ ]:
import importlib
# available class names:
# - BertModel, BertForPreTraining, BertForMaskedLM, BertForNextSentencePrediction,
# - BertForSequenceClassification, BertForMultipleChoice, BertForTokenClassification,
# - BertForQuestionAnswering
# check https://huggingface.co/docs/transformers/model_doc/bert
cls = getattr(importlib.import_module(gena_module_name), 'BertForSequenceClassification')
cls

transformers_modules.AIRI-Institute.gena-lm-bert-base-t2t.21343b983208dd7bd3430f5a0d812ab6131faa7d.modeling_bert.BertForSequenceClassification

Now we initialize the model from the pre-trained GENA-LM with classification head. We set the number of labels for the task `num_labels` to 2.

In [ ]:
model = cls.from_pretrained('AIRI-Institute/gena-lm-bert-base-t2t', num_labels=2)
print('\nclassification head:', model.classifier)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at AIRI-Institute/gena-lm-bert-base-t2t and are newly initialized: ['bert.pooler.dense.weight', 'classifier.weight', 'bert.pooler.dense.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



classification head: Linear(in_features=768, out_features=2, bias=True)


We can see that some parameters (classification head) were newly initialized: ['bert.pooler.dense.weight', 'classifier.weight', 'bert.pooler.dense.bias', 'classifier.bias']. This is fine because we will train these parameters soon.

#### alternative way with cloning the GENA-LM repo




In [ ]:
! git clone https://github.com/AIRI-Institute/GENA_LM.git
! cd GENA_LM/src/gena_lm

Cloning into 'GENA_LM'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 58 (delta 14), reused 42 (delta 6), pack-reused 0
Unpacking objects: 100% (58/58), 21.52 MiB | 8.07 MiB/s, done.


or just download `modeling_bert.py` from https://github.com/AIRI-Institute/GENA_LM/tree/main/src/gena_lm

In [ ]:
! wget https://raw.githubusercontent.com/AIRI-Institute/GENA_LM/main/src/gena_lm/modeling_bert.py

--2023-07-06 07:35:15--  https://raw.githubusercontent.com/AIRI-Institute/GENA_LM/main/src/gena_lm/modeling_bert.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94428 (92K) [text/plain]
Saving to: ‘modeling_bert.py’

modeling_bert.py    100%[===================>]  92.21K  --.-KB/s    in 0.01s   

2023-07-06 07:35:16 (6.61 MB/s) - ‘modeling_bert.py’ saved [94428/94428]



In [ ]:
from modeling_bert import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('AIRI-Institute/gena-lm-bert-base-t2t', num_labels=2)
model = model.cuda()
model.classifier

Some weights of the model checkpoint at AIRI-Institute/gena-lm-bert-base-t2t were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at AIRI-Ins

Linear(in_features=768, out_features=2, bias=True)

## Get sequence classification dataset
Here we use 11k random samples from the promoter prediction task as an example task. Given an input sequence, we have to decide whether there is a promoter or not. The dataset consists of pairs sequence and label (promoter presence - 0 or 1).

Check how to build full dataset: https://github.com/AIRI-Institute/GENA_LM/tree/main/downstream_tasks/promoter_prediction#dataset-preparation

In [ ]:
from datasets import load_dataset
# load ~11k samples from promoters prediction dataset
dataset = load_dataset("yurakuratov/example_promoters_300")['train'].train_test_split(test_size=0.1)

print(dataset)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'sequence', 'promoter_presence'],
        num_rows: 10656
    })
    test: Dataset({
        features: ['Unnamed: 0', 'sequence', 'promoter_presence'],
        num_rows: 1184
    })
})


Here is the first sample from the training set:

In [ ]:
dataset['train'][0]

{'Unnamed: 0': 10711,
 'sequence': 'CTGGACTCGGGTGTGCGGGGCTGTGAGGGGGCGAGGCGGGCGCTGCGGCCCGCCCGGGATGGGCCAGCCCTGGCCTGGCGGGGCTGAGCCGAAGGCGAGGGCGGCGCGCGGGCCAGGCCTGCCGGGCGGGCGGCCCGGGGGTTGAGGTAGAAGTGGGCGCGGAGGAAGGGGCCGAGCCAAGGCGGTGGGTGGAGCGGCGAGGGGGGCGGAGGCTGGGCCGCGGCGGGCGAGCGGAGCGGCGCGCCTGTCCGGAGCTCGGCGGTGGCGCCGGAGGAGGCTGCAGCGGCGGCGGCGGCGGGC',
 'promoter_presence': 1}

Length in base pairs:

In [ ]:
print('# base pairs: ', len(dataset['train'][0]['sequence']))

# base pairs:  300


Convert sequence to tokens:

In [ ]:
print('tokens: ', ' '.join(tokenizer.tokenize(dataset['train'][0]['sequence'])))

tokens:  C TGGACTC GGG TGTGC GGGGC TGTGAGGG GGCG AGGCGGGC GCTGC GGCCC GCCC GGG ATGGGCC AGCCC TGGCCTGGC GGGGC TGAGCCG AAGGCG AGGGCGGC GCGC GGGCC AGGCCTGCC GGGCGGGC GGCCC GGGGG TTGAGG TAGAAG TGGGCGCGG AGGAAGG GGCCG AGCCAAGGC GGTGGG TGGAGC GGCG AGGG GGGCGG AGGC TGGGCC GCGGC GGGCG AGCGG AGCGGC GCGCC TGTCCGG AGCTCGGC GGTGGC GCCGG AGGAGGC TGCAGC GGCGGCGGCGGC GGGC


Length in tokens:

In [ ]:
print('# tokens: ', len(tokenizer.tokenize(dataset['train'][0]['sequence'])))

# tokens:  51


### Dataset preprocessing
following HuggingFace text classification guide: https://huggingface.co/docs/transformers/tasks/sequence_classification

In [ ]:
def preprocess_labels(example):
  example['label'] = example['promoter_presence']
  return example

dataset = dataset.map(preprocess_labels)

Map:   0%|          | 0/10656 [00:00<?, ? examples/s]

Map:   0%|          | 0/1184 [00:00<?, ? examples/s]

In [ ]:
def preprocess_function(examples):
  # just truncate right, but for some tasks symmetric truncation from left and right is more reasonable
  # set max_length to 128 tokens to make experiments faster
  return tokenizer(examples["sequence"], truncation=True, max_length=128)

Pre-tokenize our dataset:

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/10656 [00:00<?, ? examples/s]

Map:   0%|          | 0/1184 [00:00<?, ? examples/s]

Now create a batch of examples using DataCollatorWithPadding. It’s more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Finally, we prepared our dataset and ready to fine-tune GENA-LM model on it. You can see that is has all required by the model fields: `label`, `input_ids`, `token_type_ids`, `attention_mask`.

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'sequence', 'promoter_presence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10656
    })
    test: Dataset({
        features: ['Unnamed: 0', 'sequence', 'promoter_presence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1184
    })
})

## Training

We use Trainer from the Transformers library, which will train our model on the prepared dataset. All we need is to define metrics (`compute_metrics` function) and specify training hyperparameters in `TrainingArguments`.

Let's run:

In [ ]:
from transformers import TrainingArguments, Trainer
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': (predictions==labels).sum() / len(labels)}

# change training hyperparameters to archive better quality
training_args = TrainingArguments(
    output_dir="test_run",
    learning_rate=2e-05,
    lr_scheduler_type="constant_with_warmup",
    warmup_ratio=0.1,
    optim='adamw_torch',
    weight_decay=0.0,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:900: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.707200,0.704476,0.478885
2,0.706600,0.691664,0.517736
3,0.692000,0.679301,0.601351
4,0.632100,0.609850,0.696791
5,0.545200,0.509010,0.758446
6,0.472700,0.524267,0.760135
7,0.426900,0.550644,0.761824
8,0.374800,0.539974,0.777027


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:900: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:900: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:900: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:900: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:900: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_uti

Epoch,Training Loss,Validation Loss,Accuracy
1,0.707200,0.704476,0.478885
2,0.706600,0.691664,0.517736
3,0.692000,0.679301,0.601351
4,0.632100,0.609850,0.696791
5,0.545200,0.509010,0.758446
6,0.472700,0.524267,0.760135
7,0.426900,0.550644,0.761824
8,0.374800,0.539974,0.777027
9,0.329900,0.778484,0.726351
10,0.296900,0.571850,0.794764


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:900: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


TrainOutput(global_step=3330, training_loss=0.5184220368439729, metrics={'train_runtime': 1444.8652, 'train_samples_per_second': 73.751, 'train_steps_per_second': 2.305, 'total_flos': 3305184632991360.0, 'train_loss': 0.5184220368439729, 'epoch': 10.0})

Training should take about 10-20 minutes on GPU. You can stop it any time. It is possible to get better results by tweaking hyperparameters and/or training longer.

## Get predictions from model on single example

In case you don't want to finish the training process. You can download a model that is already fine-tuned on this task.

We have made some fine-tuned GENA-LM models publicly available: https://github.com/AIRI-Institute/GENA_LM#pre-trained-models-on-downstream-tasks

In [ ]:
# optional, uncomment to download model that was already finetuned on this task:
# model = AutoModel.from_pretrained(f'AIRI-Institute/gena-lm-bert-base-t2t', revision='promoters_300_run_1', trust_remote_code=True)

Get sequence from test set and its label:

In [ ]:
x, y = dataset['test']['sequence'][0], dataset['test']['label'][0]

Preprocess sequence with tokenizer:

In [ ]:
x_feat = tokenizer(x, return_tensors='pt')
x_feat.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
# move sample to gpu and feed it to the model
for k in x_feat:
  x_feat[k] = x_feat[k].cuda()

model = model.eval()
with torch.no_grad():
  out = model(**x_feat)
out

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:900: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


SequenceClassifierOutput(loss=None, logits=tensor([[1.4964, 0.1714]], device='cuda:0'), hidden_states=None, attentions=None)

Model returned `out` with logits. We need to feed the logits into the softmax function to get probabilities:

In [ ]:
# get class probabilities
prob = torch.softmax(out['logits'], dim=-1)
prob

tensor([[0.7900, 0.2100]], device='cuda:0')

We can get label predicted by the model as label with max probability. Let's compare predicted label and true labels from the dataset.

In [ ]:
# get label
print(f'prediction: {torch.argmax(prob)}, label: {y}')

prediction: 0, label: 0


## Challenge 1: predict promoter activity for your sequence of choice
Step 1. Use this link to visualize human genomic data:
http://genome.ucsc.edu/cgi-bin/hgTracks?db=hg38&position=lastDbPos

Step 2. Right-click on the region of interest (genes are shown as blue lines) and select "Get DNA" from menu (hint: use 300-bp sequence input)

(e.g., region https://genome-asia.ucsc.edu/cgi-bin/hgc?o=87566244&g=getDna&i=chr7&c=chr7&l=87566244&r=87566247&db=hg38)

Step 3. Update this notebook to predict promoters activity of the sequence and report the results!